<a href="https://colab.research.google.com/github/bidhan7/Data-Processing-Visualization-/blob/main/Big_Query_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bidhan Subedi
## Data Visiualization



## SQL with BigQuery
The first thing, most of us already have an account at Cloud Resource Manager if you already have gmail logged in your computer. I required to do each of the following steps successfully in getting this document to work.  

## Before you begin


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Now that I am authenticated, I can start to playing around with the dataset.  I am going to look at the bikers trip data from Austin. Yes, I have a project called `data-visualization-338723` in my BigQuery interface.

## We will use SQL to compute from dataset

`SELECT` picks the columns and `FROM` picks the table you need. 

### Computing the no of entries in the dataset.

In [2]:
%%bigquery --project data-visualization-338723
SELECT 
  COUNT(*) as total_entries
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`

,total_entries
0,1424786


That's is a lot of entries.

As there is alot of entries in the data set excel will have hard time computing. SQL will do the data manipulations on the database server much better.

 

### Computing The Longest Trip

In [3]:
%%bigquery --project data-visualization-338723
SELECT 
  MAX(duration_minutes) as Longest_trip_in_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`

,Longest_trip_in_minutes
0,34238


The longest trip was 34238 minutes!

### Computing the average time for a trip based on starting point

In [5]:
%%bigquery --project data-visualization-338723
SELECT 
  start_station_name, 
  AVG(duration_minutes) as Average_time_for_trip
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name is not null
GROUP BY start_station_name

,start_station_name,Average_time_for_trip
0,Zilker Park West,27.168259
1,Toomey Rd @ South Lamar,28.115798
2,State Capitol @ 14th & Colorado,30.421842
3,Waller & 6th St.,23.265843
4,Pease Park,28.648889
...,...,...
188,Republic Square @ Guadalupe & 4th St.,23.067313
189,3rd & West,19.937466
190,3rd/West,34.152547
191,Nueces & 26th,16.805353


There is 13967 rows in above dataset. That means there are that many number of  trips. Last column on right side of the table represents average time for every indiviual trip.

#### This is just me playing around .

In [6]:
%%bigquery --project data-visualization-338723
SELECT 
  COUNT(start_station_name) as total_trips
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`

,total_trips
0,1424786


### Computing how many trips start at each starting point

In [16]:
%%bigquery --project data-visualization-338723
SELECT 
  start_station_name,
  COUNT(*) as total_trips
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
ORDER BY total_trips DESC

,start_station_name,total_trips
0,21st & Speedway @PCL,72799
1,Riverside @ S. Lamar,40635
2,City Hall / Lavaca & 2nd,36520
3,2nd & Congress,35307
4,Rainey St @ Cummings,34758
...,...,...
188,Marketing Event,4
189,Eeyore's 2018,2
190,Stolen,1
191,Eeyore's 2017,1
